# Magnetic energy and anisotropy in single-domain particles

Magnetic particles adopt magnetization states that minimize their total magnetic energy.
While the lowest-energy state is often the global minimum, particles may also occupy
**local energy minima (LEMs)** that can persist for long periods of time in the absence of sufficient
energy to overcome barriers.

The stability of a magnetization state, and the ease with which it can be changed by an external
magnetic field, are controlled by the particle's **anisotropy energy constant** $K_u$.

In this notebook we explore how anisotropy energy and magnetic interaction energy combine to control
the behavior of a single-domain, uniaxial particle in the presence of external applied fields.

## Geometry of a Uniaxial Particle (Panel a)

The following sketch defines the geometry used throughout this notebook:

- A single-domain particle with a single **easy axis** (uniaxial anisotropy).
- The magnetization vector $\mathbf{M}$ makes an angle $\theta$ with the easy axis.
- An external magnetic field $\mathbf{B}$ is applied at an angle $\phi$ to the easy axis.

<div style="text-align: center;">
  <img
    src="https://raw.githubusercontent.com/Institute-for-Rock-Magnetism/2026_ESCI_pmag_course/refs/heads/main/W3_anisotropy_hysteresis/images/magnetite_w_easy_axis.svg"
    alt="Geometry of a uniaxial particle showing θ and φ"
    width="350"
  >
</div>

In this illustration:
- $\theta$ is the angle between the magnetization direction and the easy axis,
- $\phi$ is the angle between the applied field and the easy axis.

## Shape anisotropy energy density $\epsilon_a(\theta)$

For a uniaxial particle, the anisotropy energy density is

$\epsilon_a = K_u \sin^2 \theta$.

This expression has two equivalent minima at $\theta = 0^\circ$ and $180^\circ$, corresponding to
magnetization parallel or antiparallel to the easy axis.

For shape anisotropy, the anisotropy constant $K_u$ arises from the difference in demagnetizing
factors along and perpendicular to the easy axis:

$K_u = \tfrac{1}{2} \mu_0 (N_b - N_a) M_s^2$,

where:
- $N_a$ is the demagnetizing factor along the easy axis,
- $N_b$ is the demagnetizing factor perpendicular to the easy axis.

In the absence of an applied field, the magnetization will lie along one of the easy-axis directions.

**Let's calculate $K_u$ and then $\epsilon_a$.**

### Import our tools

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = "retina"

### Calculate $K_u$ and $\epsilon_a$

In [ ]:
# Constants
mu0 = 4 * np.pi * 1e-7
Ms = 480e3  # A/m (magnetite)

# Geometry
theta_deg = np.linspace(0, 180, 181)
theta = np.deg2rad(theta_deg)

# Shape anisotropy: prolate spheroid with elongation q = a/b
q = 2.0

def demag_factors_prolate(q):
    """Demagnetizing factors for a prolate spheroid.

    Returns
    -------
    N_a : float
        Demagnetizing factor along the easy axis.
    N_b : float
        Demagnetizing factor perpendicular to the easy axis.
    """
    e = np.sqrt(1 - 1 / q**2)
    N_a = (1 - e**2) / (2 * e**3) * (np.log((1 + e) / (1 - e)) - 2 * e)
    N_b = (1 - N_a) / 2
    return N_a, N_b

N_a, N_b = demag_factors_prolate(q)

# Shape anisotropy constant
K_u = 0.5 * mu0 * (N_b - N_a) * Ms**2

# Anisotropy energy density
epsilon_a = K_u * np.sin(theta)**2

In [ ]:
theta_deg

In [ ]:
epsilon_a

### Let's plot it up

In [ ]:
plt.figure(figsize=(6, 3))
plt.plot(theta_deg, epsilon_a, linewidth=2.5, label=r"$\epsilon_a$")

plt.xlim(0, 180)
plt.xlabel(r"$\theta$ (degrees)")
plt.ylabel(r"Energy density (J m$^{-3}$)")
plt.title(r"$\epsilon_a(\theta)$ (shape anisotropy)")
plt.grid(True)

plt.legend()
plt.show()

## Magnetostatic interaction energy density $\epsilon_m(\theta)$

When an external magnetic field $\mathbf{B}$ is applied, the magnetostatic interaction energy density
is given by the dot product of the magnetization and the applied field:

$\epsilon_m = -\mathbf{M} \cdot \mathbf{B}
            = - M_s B \cos(\phi - \theta)$.

This term favors alignment of the magnetization with the applied field direction.

In [ ]:
# Applied field
B_mT = 30.0
B = B_mT * 1e-3  # Tesla
phi_deg = 45.0
phi = np.deg2rad(phi_deg)

# Interaction energy density
epsilon_m = -Ms * B * np.cos(phi - theta)

In [ ]:
plt.figure(figsize=(6, 3))
plt.plot(theta_deg, epsilon_m, "--", linewidth=2.5, label=r"$\epsilon_m$",
         color='C1')

plt.xlim(0, 180)
plt.xlabel(r"$\theta$ (degrees)")
plt.ylabel(r"Energy density (J m$^{-3}$)")
plt.title(rf"$\epsilon_m(\theta)$ ($B={B_mT}$ mT, $\phi={phi_deg}^\circ$)")
plt.grid(True)

plt.legend()
plt.show()

## Total energy density $\epsilon_t(\theta)$ and local energy minima

Assuming the particle is magnetized at saturation, the total energy density is

$\epsilon_t = \epsilon_a + \epsilon_m
            = K_u \sin^2\theta - M_s B \cos(\phi - \theta)$.

The magnetization of a single-domain particle will adopt the angle $\theta$ that minimizes
$\epsilon_t$.

- At low applied fields, the minimum lies close to the easy axis.
- At higher applied fields, the minimum shifts toward the applied field direction.
- Local energy minima may persist even when they are not the global minimum.

The disappearance of a local energy minimum is what leads to irreversible switching
(the *flipping field*).

In [ ]:
epsilon_t = epsilon_a + epsilon_m

Let's find the minimum total energy

In [ ]:
i_min = np.argmin(epsilon_t)
epsilon_t[i_min]

In [ ]:
plt.figure(figsize=(6, 3))

plt.plot(theta_deg, epsilon_a, linewidth=2.5, label=r"$\epsilon_a$")
plt.plot(theta_deg, epsilon_m, "--", linewidth=2.5, label=r"$\epsilon_m$")
plt.plot(
    theta_deg,
    epsilon_t,
    linewidth=4.0,
    label=r"$\epsilon_t=\epsilon_a+\epsilon_m$",
)

# --- mark epsilon_min (already computed) ---
plt.scatter(
    theta_deg[i_min],
    epsilon_t[i_min],
    s=70,
    facecolors="none",
    edgecolors="k",
    zorder=5,
)
plt.text(
    theta_deg[i_min] + 5,
    epsilon_t[i_min] + 0.05 * (epsilon_t.max() - epsilon_t.min()),
    r"$\epsilon_{\min}$",
    fontsize=12,
)

plt.xlim(0, 180)
plt.xticks(np.arange(0, 181, 20))
plt.xlabel(r"$\theta$ (degrees)")
plt.ylabel(r"Energy density (J m$^{-3}$)")
plt.title(rf"$\epsilon_t(\theta)$ ($B={B_mT}$ mT, $\phi={phi_deg}^\circ$)")
plt.grid(True)

plt.legend()
plt.show()

In [ ]:
# @title Putting it all together for a new Essentials Figure { display-mode: "form" }

import io
import urllib.request

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np

# If you want "retina" inline figures in Jupyter, run (in its own cell):
# %config InlineBackend.figure_format = "retina"

# ---------------------------
# Panel (a): load PNG from GitHub
# ---------------------------
panel_a_url = (
    "https://github.com/Institute-for-Rock-Magnetism/2026_ESCI_pmag_course/raw/refs/heads/main/"
    "W3_anisotropy_hysteresis/images/magnetite_w_easy_axis.png"
)

with urllib.request.urlopen(panel_a_url) as resp:
    panel_a_img = mpimg.imread(io.BytesIO(resp.read()))

# ---------------------------
# Model setup (self-contained)
# ---------------------------
# Constants
mu0 = 4 * np.pi * 1e-7
M_s = 480e3  # A/m (magnetite)

# Angle grid
theta_deg = np.linspace(0.0, 180.0, 1801)  # fine grid for smooth curves
theta = np.deg2rad(theta_deg)

# Shape anisotropy: prolate spheroid with elongation q = a/b
q = 2.0


def demag_factors_prolate(q: float) -> tuple[float, float]:
    """Demagnetizing factors for a prolate spheroid.

    Parameters
    ----------
    q : float
        Elongation ratio a/b (>1 for prolate).

    Returns
    -------
    tuple[float, float]
        (N_a, N_b) where N_a is along the symmetry/easy axis and
        N_b is perpendicular (with N_b = N_c for a spheroid).
    """
    if q <= 1:
        raise ValueError("q must be > 1 for a prolate spheroid.")

    e = np.sqrt(1.0 - 1.0 / q**2)
    N_a = (1.0 - e**2) / (2.0 * e**3) * (np.log((1.0 + e) / (1.0 - e)) - 2.0 * e)
    N_b = (1.0 - N_a) / 2.0
    return float(N_a), float(N_b)


N_a, N_b = demag_factors_prolate(q)

# Shape-anisotropy constant (Tauxe: K_u = 1/2 mu0 (N_b - N_a) M_s^2)
K_u = 0.5 * mu0 * (N_b - N_a) * M_s**2

# Energy densities
epsilon_a = K_u * np.sin(theta) ** 2  # anisotropy energy density

# Panel (b): B = 0 mT (only epsilon_a matters)
phi_deg = 45.0
epsilon_a_0 = epsilon_a
i_min_b = int(np.argmin(epsilon_a_0))

# Panel (c): finite B
B_mT = 30.0
B = B_mT * 1e-3  # Tesla
phi = np.deg2rad(phi_deg)

epsilon_m = -M_s * B * np.cos(phi - theta)  # interaction energy density
epsilon_t = epsilon_a + epsilon_m           # total energy density
i_min_c = int(np.argmin(epsilon_t))

# ---------------------------
# Figure layout
# ---------------------------
fig = plt.figure(figsize=(12, 6))
gs = fig.add_gridspec(
    nrows=2,
    ncols=2,
    width_ratios=[1.0, 1.9],
    height_ratios=[1.0, 1.0],
    wspace=0.25,
    hspace=0.30,
)

ax_a = fig.add_subplot(gs[:, 0])
ax_b = fig.add_subplot(gs[0, 1])
ax_c = fig.add_subplot(gs[1, 1])

# ---------------- Panel (a) ----------------
ax_a.imshow(panel_a_img)
ax_a.axis("off")
ax_a.text(
    0.02,
    0.98,
    "a)",
    transform=ax_a.transAxes,
    va="top",
    ha="left",
    fontsize=16,
    fontweight="bold",
)

# Common axis styling
def style_axes(ax: plt.Axes) -> None:
    ax.set_xlim(0, 180)
    ax.set_xticks(np.arange(0, 181, 20))
    ax.tick_params(direction="in", length=6, width=1)
    for spine in ax.spines.values():
        spine.set_linewidth(1.8)
    ax.grid(True)


# ---------------- Panel (b) ----------------
ax_b.plot(theta_deg, epsilon_a_0, linewidth=2.5, label=r"$\epsilon_a$")
ax_b.axhline(0.0, linewidth=1)
ax_b.set_ylabel(r"Energy density (J m$^{-3}$)")
ax_b.set_title(rf"b) $B = 0$ mT ($\phi = {phi_deg:.0f}^\circ$)")

ax_b.scatter(
    theta_deg[i_min_b],
    epsilon_a_0[i_min_b],
    s=70,
    facecolors="none",
    edgecolors="k",
    zorder=5,
)
ax_b.text(
    theta_deg[i_min_b] + 5,
    epsilon_a_0[i_min_b] + 0.05 * (epsilon_a_0.max() - epsilon_a_0.min()),
    r"$\epsilon_{\min}$",
    fontsize=12,
)

style_axes(ax_b)
ax_b.legend(loc="best", frameon=True)

# ---------------- Panel (c) ----------------
ax_c.plot(theta_deg, epsilon_a, linewidth=2.5, label=r"$\epsilon_a$")
ax_c.plot(theta_deg, epsilon_m, "--", linewidth=2.5, label=r"$\epsilon_m$")
ax_c.plot(
    theta_deg,
    epsilon_t,
    linewidth=4.0,
    label=r"$\epsilon_t=\epsilon_a+\epsilon_m$",
)

ax_c.axhline(0.0, linewidth=1)
ax_c.set_xlabel(r"$\theta$ (degrees)")
ax_c.set_ylabel(r"Energy density (J m$^{-3}$)")
ax_c.set_title(rf"c) $B = {B_mT:.0f}$ mT ($\phi = {phi_deg:.0f}^\circ$)")

ax_c.scatter(
    theta_deg[i_min_c],
    epsilon_t[i_min_c],
    s=70,
    facecolors="none",
    edgecolors="k",
    zorder=5,
)
ax_c.text(
    theta_deg[i_min_c] + 5,
    epsilon_t[i_min_c] + 0.05 * (epsilon_t.max() - epsilon_t.min()),
    r"$\epsilon_{\min}$",
    fontsize=12,
)

style_axes(ax_c)
ax_c.legend(loc="best", frameon=True)

plt.show()

## Visualizing the flipping field

This interactive widget builds directly on the energy framework developed above to show **how and when a single-domain particle actually switches its magnetization** in response to an applied field. This is the flipping (switching) field.

### Geometry and field history

In this visualization we consider:

- A single-domain particle with **uniaxial (shape) anisotropy**.
- The applied field **$\mathbf{B}$ is parallel to the easy axis** ($\phi = 0^\circ$).
- The magnetization **starts antiparallel to the field** ($\theta = 180^\circ$), representing a remanent state.

### Metastability and local energy minima

Although the total energy $\epsilon_t(\theta)$ may already have a lower (global) minimum at small applied fields, the magnetization does **not** immediately jump to it.

Instead:

- The magnetization remains trapped in the **local energy minimum** that is continuously connected to its previous state.
- This local minimum represents a **metastable configuration** separated from the global minimum by an energy barrier.
- The flipping field is reached when this occupied local minimum **ceases to exist**.

This loss of the local minimum is what forces the magnetization to switch irreversibly.

### Why we look at derivatives

To diagnose when a local minimum disappears, the widget shows the angular derivatives of the total energy:

- The **first derivative**
  $$
  \frac{d\epsilon_t}{d\theta}
  $$
  identifies stationary points.

- The **second derivative**
  $$
  \frac{d^2\epsilon_t}{d\theta^2}
  $$
  determines whether a stationary point is stable (positive curvature) or unstable.

The **flipping condition** occurs when the occupied stationary point satisfies:
- $\frac{d\epsilon_t}{d\theta} = 0$  
- $\frac{d^2\epsilon_t}{d\theta^2} = 0$

At this point, the local minimum merges with a saddle point and vanishes.

In [ ]:
# @title The flipping widget { display-mode: "form" }

from __future__ import annotations

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display


def make_flipping_stepper(
    K_u: float,
    M_s: float,
    phi_deg: float = 0.0,
    theta_init_deg: float = 180.0,
    b_max_mT: float | None = None,
    b_step_mT: float = 1.0,
    theta_step_deg: float = 0.05,
    track_window_deg: float = 30.0,
) -> widgets.VBox:
    """Fast SW flipping visualization with step buttons + text entry.

    Uses incremental state updates (no recomputing from B=0 each time).
    """
    phi = np.deg2rad(phi_deg)

    b_k_T = 2.0 * K_u / M_s
    b_k_mT = b_k_T * 1e3
    if b_max_mT is None:
        b_max_mT = float(1.5 * b_k_mT)

    theta_deg = np.arange(0.0, 180.0 + theta_step_deg, theta_step_deg)
    theta = np.deg2rad(theta_deg)

    def eps_a(theta_arr: np.ndarray) -> np.ndarray:
        return K_u * np.sin(theta_arr) ** 2

    def eps_m(theta_arr: np.ndarray, b_T: float) -> np.ndarray:
        return -M_s * b_T * np.cos(phi - theta_arr)

    def eps_t(theta_arr: np.ndarray, b_T: float) -> np.ndarray:
        return eps_a(theta_arr) + eps_m(theta_arr, b_T)

    def deps_dtheta(theta_arr: np.ndarray, b_T: float) -> np.ndarray:
        return (
            2.0 * K_u * np.sin(theta_arr) * np.cos(theta_arr)
            - M_s * b_T * np.sin(phi - theta_arr)
        )

    def d2eps_dtheta2(theta_arr: np.ndarray, b_T: float) -> np.ndarray:
        return 2.0 * K_u * np.cos(2.0 * theta_arr) + M_s * b_T * np.cos(phi - theta_arr)

    def style_axes(ax: plt.Axes) -> None:
        ax.set_xlim(0, 180)
        ax.set_xticks(np.arange(0, 181, 20))
        ax.tick_params(direction="in", length=6, width=1)
        for spine in ax.spines.values():
            spine.set_linewidth(1.8)
        ax.grid(True)

    def find_stable_stationary_near(center_deg: float, window_deg: float, b_T: float) -> int | None:
        d1 = deps_dtheta(theta, b_T)
        d2 = d2eps_dtheta2(theta, b_T)

        mask = (theta_deg >= center_deg - window_deg) & (theta_deg <= center_deg + window_deg)
        idx = np.where(mask)[0]
        if idx.size < 2:
            return None

        candidates: list[int] = []

        # Endpoints
        for i_end in (0, len(theta_deg) - 1):
            if (theta_deg[i_end] >= center_deg - window_deg) and (
                theta_deg[i_end] <= center_deg + window_deg
            ):
                if np.isfinite(d1[i_end]) and np.isfinite(d2[i_end]):
                    if (np.abs(d1[i_end]) < 1e-6 * K_u) and (d2[i_end] > 0):
                        candidates.append(i_end)

        # Interior roots of d1
        d1_sub = d1[idx]
        finite = np.isfinite(d1_sub)
        idx_f = idx[finite]
        if idx_f.size >= 2:
            d1_f = d1[idx_f]
            s = np.sign(d1_f)
            sc = np.where(s[:-1] * s[1:] < 0)[0]

            for k in sc:
                i0 = idx_f[k]
                i1 = idx_f[k + 1]
                denom = d1[i0] - d1[i1]
                if np.abs(denom) < 1e-30:
                    continue
                t = d1[i0] / denom
                th_root = theta[i0] + t * (theta[i1] - theta[i0])
                i_root = int(np.argmin(np.abs(theta - th_root)))
                if (theta_deg[i_root] >= center_deg - window_deg) and (
                    theta_deg[i_root] <= center_deg + window_deg
                ):
                    if np.isfinite(d2[i_root]) and (d2[i_root] > 0):
                        candidates.append(i_root)

        if not candidates:
            return None

        candidates = sorted(set(candidates))
        return int(min(candidates, key=lambda i: abs(theta_deg[i] - center_deg)))

    # --------- incremental state ---------
    state = {
        "b_mT": 0.0,
        "theta_occ_deg": float(theta_init_deg),
        "flipped": False,
    }

    b_box = widgets.BoundedFloatText(
        value=0.0,
        min=0.0,
        max=float(b_max_mT),
        step=float(b_step_mT),
        description="B (mT):",
        style={"description_width": "80px"},
        layout=widgets.Layout(width="240px"),
    )

    btn_m5 = widgets.Button(description="−5", layout=widgets.Layout(width="60px"))
    btn_m1 = widgets.Button(description="−1", layout=widgets.Layout(width="60px"))
    btn_p1 = widgets.Button(description="+1", layout=widgets.Layout(width="60px"))
    btn_p5 = widgets.Button(description="+5", layout=widgets.Layout(width="60px"))
    btn_reset = widgets.Button(description="Reset", layout=widgets.Layout(width="90px"))

    info = widgets.HTML()
    out = widgets.Output()

    def redraw() -> None:
        b_mT = state["b_mT"]
        b_T = b_mT * 1e-3

        ea = eps_a(theta)
        em = eps_m(theta, b_T)
        et = ea + em

        d1 = deps_dtheta(theta, b_T) / K_u
        d2 = d2eps_dtheta2(theta, b_T) / K_u

        i_global = int(np.argmin(et))
        i_occ = find_stable_stationary_near(state["theta_occ_deg"], track_window_deg, b_T)

        status = "flipped" if state["flipped"] else "not flipped (trapped in LEM)"
        info.value = (
            f"<b>Geometry:</b> φ = {phi_deg:.0f}°, start θ = {theta_init_deg:.0f}°<br>"
            f"<b>Tracked state:</b> θ ≈ {state['theta_occ_deg']:.2f}° → <b>{status}</b>"
        )

        with out:
            out.clear_output(wait=True)
            fig, axes = plt.subplots(1, 2, figsize=(12, 3.4), sharex=True)

            ax = axes[0]
            ax.plot(theta_deg, ea, linewidth=2.5, label=r"$\epsilon_a$")
            ax.plot(theta_deg, em, "--", linewidth=2.5, label=r"$\epsilon_m$")
            ax.plot(theta_deg, et, linewidth=4.0, label=r"$\epsilon_t=\epsilon_a+\epsilon_m$")
            ax.axhline(0.0, linewidth=1)

            ax.scatter(theta_deg[i_global], et[i_global], s=70, facecolors="none", edgecolors="k")
            ax.text(theta_deg[i_global] + 4, et[i_global], "global", fontsize=10, va="center")

            if i_occ is not None:
                ax.scatter(
                    theta_deg[i_occ],
                    et[i_occ],
                    s=120,
                    facecolors="none",
                    edgecolors="red",
                    linewidths=2,
                )
                ax.text(
                    theta_deg[i_occ] + 4,
                    et[i_occ],
                    "occupied\nLEM",
                    fontsize=10,
                    color="red",
                    va="center",
                )
            else:
                ax.text(
                    0.03,
                    0.12,
                    "occupied LEM has disappeared → flip",
                    transform=ax.transAxes,
                    color="red",
                    fontsize=11,
                )

            ax.set_ylabel(r"Energy density (J m$^{-3}$)")
            ax.set_title(rf"Energies ($B={b_mT:.0f}$ mT, $\phi={phi_deg:.0f}^\circ$)")
            style_axes(ax)
            ax.legend(loc="best", frameon=True)

            ax = axes[1]
            ax.plot(theta_deg, d1, linewidth=2.5, label=r"$\frac{1}{K_u}\frac{d\epsilon_t}{d\theta}$")
            ax.plot(theta_deg, d2, "--", linewidth=2.5, label=r"$\frac{1}{K_u}\frac{d^2\epsilon_t}{d\theta^2}$")
            ax.axhline(0.0, linewidth=1)

            ax.axvline(state["theta_occ_deg"], linewidth=1.8)
            ax.set_ylabel("Normalized derivatives")
            ax.set_title("Flipping condition diagnostic")
            style_axes(ax)
            ax.legend(loc="best", frameon=True)

            axes[0].set_xlabel(r"$\theta$ (degrees)")
            axes[1].set_xlabel(r"$\theta$ (degrees)")
            plt.tight_layout()
            plt.show()

    def step_to(new_b_mT: float) -> None:
        new_b_mT = float(np.clip(new_b_mT, 0.0, b_max_mT))

        # Incrementally walk from current B to new B in steps of b_step_mT
        b0 = state["b_mT"]
        if np.isclose(new_b_mT, b0):
            return

        direction = 1.0 if new_b_mT > b0 else -1.0
        steps = np.arange(b0, new_b_mT + 0.5 * direction * b_step_mT, direction * b_step_mT)

        # If stepping backward, reset and replay forward (hysteresis needs path memory)
        if direction < 0:
            state["b_mT"] = 0.0
            state["theta_occ_deg"] = float(theta_init_deg)
            state["flipped"] = False
            steps = np.arange(0.0, new_b_mT + 0.5 * b_step_mT, b_step_mT)

        for b_mT in steps[1:]:
            b_T = b_mT * 1e-3
            e = eps_t(theta, b_T)

            i_occ = find_stable_stationary_near(state["theta_occ_deg"], track_window_deg, b_T)
            if i_occ is None:
                i_global = int(np.argmin(e))
                state["theta_occ_deg"] = float(theta_deg[i_global])
                state["flipped"] = True
            else:
                state["theta_occ_deg"] = float(theta_deg[i_occ])

            state["b_mT"] = float(b_mT)

        b_box.value = state["b_mT"]
        redraw()

    def on_box_change(change) -> None:
        if change["name"] == "value":
            step_to(float(change["new"]))

    b_box.observe(on_box_change, names="value")

    def on_click(delta: float) -> None:
        step_to(state["b_mT"] + delta)

    btn_m5.on_click(lambda _: on_click(-5.0))
    btn_m1.on_click(lambda _: on_click(-1.0))
    btn_p1.on_click(lambda _: on_click(+1.0))
    btn_p5.on_click(lambda _: on_click(+5.0))

    def on_reset(_):
        state["b_mT"] = 0.0
        state["theta_occ_deg"] = float(theta_init_deg)
        state["flipped"] = False
        b_box.value = 0.0
        redraw()

    btn_reset.on_click(on_reset)

    controls = widgets.HBox([b_box, btn_m5, btn_m1, btn_p1, btn_p5, btn_reset])
    redraw()
    return widgets.VBox([controls, info, out])


# Demo values (Tauxe magnetite example): Bk ~ 58 mT
K_u_demo = 1.4e4
M_s_demo = 480e3

w = make_flipping_stepper(
    K_u=K_u_demo,
    M_s=M_s_demo,
    phi_deg=0.0,
    theta_init_deg=180.0,
    b_step_mT=1.0,
    theta_step_deg=0.05,
    track_window_deg=30.0,
)
display(w)

### How to explore with the widget

- Step the applied field upward in small increments.
- Track the occupied local minimum in the energy plot.
- Observe how the curvature at that point decreases as the field increases.
- **Play with the widget to find the field at which the local minimum disappears and the magnetization jumps to the opposite easy-axis direction.**

We will discuss how this setup corresponds to the classic square hysteresis loop.